

---
# **consultamos desde un audio, respuesta en texto y en audio**


INSTALAR PRIMERO WHISPER Y RECONECTAR!! AHORA COMENZAMOS LA PARTE DE TEXTO-AUDIO-TEXTO.

In [ ]:
# Instalar el paquete whisperx desde el repositorio de GitHub
!pip install git+https://github.com/m-bain/whisperx.git --upgrade

In [1]:
import whisperx

device = "cuda"
batch_size = 16
compute_type = "float16"

/usr/local/lib/python3.10/dist-packages/pyannote/audio/core/io.py:43: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")


In [2]:
# Cargamos el modelo
model = whisperx.load_model("large-v2", device, compute_type=compute_type)

model.bin:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

vocabulary.txt:   0%|          | 0.00/460k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.20M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.80k [00:00<?, ?B/s]

No language specified, language will be first be detected for each audio file (increases inference time).


100%|█████████████████████████████████████| 16.9M/16.9M [00:02<00:00, 7.02MiB/s]
INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.2.0.post0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../root/.cache/torch/whisperx-vad-segmentation.bin`


Model was trained with pyannote.audio 0.0.1, yours is 3.1.1. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.1.0+cu121. Bad things might happen unless you revert torch to 1.x.


In [3]:
#Cargamos los audios
audio1 = whisperx.load_audio("/content/drive/MyDrive/Archivos/NLP/prueba1.m4a")
#audio2 = whisperx.load_audio("/content/drive/MyDrive/Archivos/NLP/AmpliarPrimero.mp3")

In [4]:
result = model.transcribe(audio1, batch_size=batch_size)
preg1=result["segments"][0]["text"]
#result = model.transcribe(audio2, batch_size=batch_size)
#preg2=result["segments"][0]["text"]

Detected language: es (0.94) in first 30s of audio...


In [5]:
print (preg1)
#print (preg2)

 ¿Quién es el profesor del curso y cuál es su currículum?


In [9]:
!pip install openai -q
!pip install langchain -q
!pip install tiktoken -q
!pip install pinecone-client -q
!pip install python-dotenv -q

In [10]:
# Montar Google Drive
import os
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
 # Carga OPENAI_API_KEY
from dotenv import load_dotenv, find_dotenv
load_dotenv('/content/drive/MyDrive/Archivos/NLP/.env')

True

In [12]:
# Conectarse con Pinecone
from pinecone import Pinecone, PodSpec
pc = Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Pinecone

In [13]:
model_name = 'text-embedding-ada-002'
OPENAI_API_KEY=os.environ.get("OPENAI_API_KEY")
indice="videos"
embedding= OpenAIEmbeddings(
    model=model_name,
    openai_api_key=OPENAI_API_KEY
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [14]:
# Cargo el los vectores desde Pienecone
vector_store=Pinecone.from_existing_index(indice,embedding)

In [15]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

In [16]:
# agregamos memoria para hacerlo conversasional
from langchain.chains import ConversationalRetrievalChain  # lo usamos para hacer una convesasion parecida a chat gpt
from langchain.chat_models import ChatOpenAI

llm=ChatOpenAI(model="gpt-3.5-turbo",temperature=1)
retriever=vector_store.as_retriever(search_type="similarity",search_kwargs={"k":3}) # modelo de recuperacion de la consulta
crc=ConversationalRetrievalChain.from_llm(llm=llm,retriever=retriever)
memoria=[]

In [17]:
pregunta=preg1
respuesta1=crc({"question":pregunta,"chat_history":memoria})
memoria.append((pregunta,respuesta1["answer"]))

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


In [18]:
print (respuesta1["answer"])

El profesor del curso es Gustavo Cabrera y su currículum indica que tiene el título de Analista Universitario de Sistemas de la Universidad Tecnológica de Córdoba, Argentina.


HASTA AQUI PARA PROBAR AUDIO A TEXTO Y QUE RESPONDA CON TEXTO

# **9) Brindar la respuesta en un archivo de audio**

In [19]:
import openai
from pathlib import Path

In [20]:
response = openai.audio.speech.create(
  model="tts-1",
  voice="alloy",
  input=respuesta1["answer"]
)

response.stream_to_file("/content/drive/MyDrive/Archivos/NLP/respuesta1.mp3")